# Analyse - Predict

Functions are important in reducing the replication of code as well as giving the user the functionality of getting an ouput on varying inputs. The functions you will write all use Eskom data/variables.

## Instructions to Students
- **Do not add or remove cells in this notebook. Do not edit or remove the `### START FUNCTION` or `### END FUNCTION` comments. Do not add any code outside of the functions you are required to edit. Doing any of this will lead to a mark of 0%!**
- Answer the questions according to the specifications provided.
- Use the given cell in each question to to see if your function matches the expected outputs.
- Do not hard-code answers to the questions.
- The use of stackoverflow, google, and other online tools are permitted. However, copying fellow student's code is not permissible and is considered a breach of the Honour code. Doing this will result in a mark of 0%.
- Good luck, and may the force be with you!

## Imports

In [1]:
import pandas as pd
import numpy as np

## Data Loading and Preprocessing

### Electricification by province (EBP) data

In [2]:
ebp_url = 'https://raw.githubusercontent.com/Explore-AI/Public-Data/master/Data/electrification_by_province.csv'
ebp_df = pd.read_csv(ebp_url)

for col, row in ebp_df.iloc[:,1:].iteritems():
    ebp_df[col] = ebp_df[col].str.replace(',','').astype(int)

ebp_df.head()

,Financial Year (1 April - 30 March),Limpopo,Mpumalanga,North west,Free State,Kwazulu Natal,Eastern Cape,Western Cape,Northern Cape,Gauteng
0,2000/1,51860,28365,48429,21293,63413,49008,48429,6168,39660
1,2001/2,68121,26303,38685,20928,64123,45773,38685,10359,36024
2,2002/3,49881,11976,28532,10316,63078,55748,28532,6869,32127
3,2003/4,42034,33515,34027,16135,60282,47414,34027,10976,39488
4,2004/5,54646,16218,21450,5668,37811,42041,21450,6316,18422


### Twitter data

In [3]:
twitter_url = 'https://raw.githubusercontent.com/Explore-AI/Public-Data/master/Data/twitter_nov_2019.csv'
twitter_df = pd.read_csv(twitter_url)
twitter_df.head()

,Tweets,Date
0,@BongaDlulane Please send an email to mediades...,2019-11-29 12:50:54
1,@saucy_mamiie Pls log a call on 0860037566,2019-11-29 12:46:53
2,@BongaDlulane Query escalated to media desk.,2019-11-29 12:46:10
3,"Before leaving the office this afternoon, head...",2019-11-29 12:33:36
4,#ESKOMFREESTATE #MEDIASTATEMENT : ESKOM SUSPEN...,2019-11-29 12:17:43


## Important Variables (Do not edit these!)

In [4]:
# gauteng ebp data as a list
gauteng = ebp_df['Gauteng'].astype(float).to_list()

# dates for twitter tweets
dates = twitter_df['Date'].to_list()

# dictionary mapping official municipality twitter handles to the municipality name
mun_dict = {
    '@CityofCTAlerts' : 'Cape Town',
    '@CityPowerJhb' : 'Johannesburg',
    '@eThekwiniM' : 'eThekwini' ,
    '@EMMInfo' : 'Ekurhuleni',
    '@centlecutility' : 'Mangaung',
    '@NMBmunicipality' : 'Nelson Mandela Bay',
    '@CityTshwane' : 'Tshwane'
}

# dictionary of english stopwords
stop_words_dict = {
    'stopwords':[
        'where', 'done', 'if', 'before', 'll', 'very', 'keep', 'something', 'nothing', 'thereupon', 
        'may', 'why', 'â€™s', 'therefore', 'you', 'with', 'towards', 'make', 'really', 'few', 'former', 
        'during', 'mine', 'do', 'would', 'of', 'off', 'six', 'yourself', 'becoming', 'through', 
        'seeming', 'hence', 'us', 'anywhere', 'regarding', 'whole', 'down', 'seem', 'whereas', 'to', 
        'their', 'various', 'thereafter', 'â€˜d', 'above', 'put', 'sometime', 'moreover', 'whoever', 'although', 
        'at', 'four', 'each', 'among', 'whatever', 'any', 'anyhow', 'herein', 'become', 'last', 'between', 'still', 
        'was', 'almost', 'twelve', 'used', 'who', 'go', 'not', 'enough', 'well', 'â€™ve', 'might', 'see', 'whose', 
        'everywhere', 'yourselves', 'across', 'myself', 'further', 'did', 'then', 'is', 'except', 'up', 'take', 
        'became', 'however', 'many', 'thence', 'onto', 'â€˜m', 'my', 'own', 'must', 'wherein', 'elsewhere', 'behind', 
        'becomes', 'alone', 'due', 'being', 'neither', 'a', 'over', 'beside', 'fifteen', 'meanwhile', 'upon', 'next', 
        'forty', 'what', 'less', 'and', 'please', 'toward', 'about', 'below', 'hereafter', 'whether', 'yet', 'nor', 
        'against', 'whereupon', 'top', 'first', 'three', 'show', 'per', 'five', 'two', 'ourselves', 'whenever', 
        'get', 'thereby', 'noone', 'had', 'now', 'everyone', 'everything', 'nowhere', 'ca', 'though', 'least', 
        'so', 'both', 'otherwise', 'whereby', 'unless', 'somewhere', 'give', 'formerly', 'â€™d', 'under', 
        'while', 'empty', 'doing', 'besides', 'thus', 'this', 'anyone', 'its', 'after', 'bottom', 'call', 
        'nâ€™t', 'name', 'even', 'eleven', 'by', 'from', 'when', 'or', 'anyway', 'how', 'the', 'all', 
        'much', 'another', 'since', 'hundred', 'serious', 'â€˜ve', 'ever', 'out', 'full', 'themselves', 
        'been', 'in', "'d", 'wherever', 'part', 'someone', 'therein', 'can', 'seemed', 'hereby', 'others', 
        "'s", "'re", 'most', 'one', "n't", 'into', 'some', 'will', 'these', 'twenty', 'here', 'as', 'nobody', 
        'also', 'along', 'than', 'anything', 'he', 'there', 'does', 'we', 'â€™ll', 'latterly', 'are', 'ten', 
        'hers', 'should', 'they', 'â€˜s', 'either', 'am', 'be', 'perhaps', 'â€™re', 'only', 'namely', 'sixty', 
        'made', "'m", 'always', 'those', 'have', 'again', 'her', 'once', 'ours', 'herself', 'else', 'has', 'nine', 
        'more', 'sometimes', 'your', 'yours', 'that', 'around', 'his', 'indeed', 'mostly', 'cannot', 'â€˜ll', 'too', 
        'seems', 'â€™m', 'himself', 'latter', 'whither', 'amount', 'other', 'nevertheless', 'whom', 'for', 'somehow', 
        'beforehand', 'just', 'an', 'beyond', 'amongst', 'none', "'ve", 'say', 'via', 'but', 'often', 're', 'our', 
        'because', 'rather', 'using', 'without', 'throughout', 'on', 'she', 'never', 'eight', 'no', 'hereupon', 
        'them', 'whereafter', 'quite', 'which', 'move', 'thru', 'until', 'afterwards', 'fifty', 'i', 'itself', 'nâ€˜t',
        'him', 'could', 'front', 'within', 'â€˜re', 'back', 'such', 'already', 'several', 'side', 'whence', 'me', 
        'same', 'were', 'it', 'every', 'third', 'together'
    ]
}

## Function 1: Metric Dictionary

Write a function that calculates the mean, median, variance, standard deviation, minimum and maximum of of list of items. You can assume the given list is contains only numerical entries, and you may use numpy functions to do this.

**Function Specifications:**
- Function should allow a list as input.
- It should return a `dict` with keys `'mean'`, `'median'`, `'std'`, `'var'`, `'min'`, and `'max'`, corresponding to the mean, median, standard deviation, variance, minimum and maximum of the input list, respectively.
- The standard deviation and variance values must be unbiased. **Hint:** use the `ddof` parameter in the corresponding numpy functions!
- All values in the returned `dict` should be rounded to 2 decimal places.

In [5]:
### START FUNCTION
def dictionary_of_metrics(items):
    """ Function that calculates the mean, median, variance, standard deviation, minimum and maximum of list in items.
       Parameters:
         items(list): takes up a list of numbers.
       Returns:
         dictionary: returning the 'mean', 'median', 'var','std','min' , 'max',
         corresponding to the mean, median, variance, standard deviation, minimum and maximum of the input list.
       Examples:
         >>>dictionary_of_metrics(gauteng)
         {'mean': 26244.42, median': 24403.5, 'var': 108160153.17, 'std': 10400.01, 'min': 8842.0,'max': 39660.0}
    """
    dic = {}
    dic['mean'] = round(np.mean(items), 2)
    dic['median'] =round(np.median(items), 2)
    dic['var'] = round(np.var(items, ddof = 1),2)
    dic['std'] =round(np.std(items, ddof = 1), 2)
    dic['min'] = round(min(items), 2)
    dic['max'] =  round(max(items), 2)

    return dic
### END FUNCTION

In [6]:
dictionary_of_metrics(gauteng) == {'mean': 26244.42,
                                   'median': 24403.5,
                                   'var': 108160153.17,
                                   'std': 10400.01,
                                   'min': 8842.0,
                                   'max': 39660.0}

True

_**Expected Output**_:

```python
dictionary_of_metrics(gauteng) == {'mean': 26244.42,
                                   'median': 24403.5,
                                   'var': 108160153.17,
                                   'std': 10400.01,
                                   'min': 8842.0,
                                   'max': 39660.0}
 ```

## Function 2: Five Number Summary

Write a function which takes in a list of integers and returns a dictionary of the [five number summary.](https://www.statisticshowto.datasciencecentral.com/how-to-find-a-five-number-summary-in-statistics/).

**Function Specifications:**
- The function should take a list as input.
- The function should return a `dict` with keys `'max'`, `'median'`, `'min'`, `'q1'`, and `'q3'` corresponding to the maximum, median, minimum, first quartile and third quartile, respectively. You may use numpy functions to aid in your calculations.
- All numerical values should be rounded to two decimal places.

In [7]:
### START FUNCTION
def five_num_summary(items):
    '''Function takes a list of integers or floats & returns a dictionary of the five number summary.
       Parameters:
         items (list): List of integers or floats
       Returns:
         summary_dict (dict): dictionary of the five number summary of list elements
       Example:
       -----------
         >>> five_num_summary([111,2221,528,588,524])
         {'max': 2221.0,
             'median': 528.0,
             'min': 111.0,
             'q1': 524.0,
             'q3': 588.0}
    '''

    summary_dict = {}
    summary = np.quantile(items,[1, 0.5, 0, 0.25, 0.75])
    list1 = ["max","median","min","q1","q3"]
    for i in range(len(list1)):
        summary_dict[list1[i]] = summary[i]

    return summary_dict
### END FUNCTION

In [9]:
five_num_summary(gauteng) == {
    'max': 39660.0,
    'median': 24403.5,
    'min': 8842.0,
    'q1': 18653.0,
    'q3': 36372.0
}

True

_**Expected Output:**_

```python
five_num_summary(gauteng) == {
    'max': 39660.0,
    'median': 24403.5,
    'min': 8842.0,
    'q1': 18653.0,
    'q3': 36372.0
}

```

## Function 3: Date Parser

The `dates` variable (created at the top of this notebook) is a list of dates represented as strings. The string contains the date in `'yyyy-mm-dd'` format, as well as the time in `hh:mm:ss` formamt. The first three entries in this variable are:
```python
dates[:3] == [
    '2019-11-29 12:50:54',
    '2019-11-29 12:46:53',
    '2019-11-29 12:46:10'
]
```

Write a function that takes as input a list of these datetime strings and returns only the date in `'yyyy-mm-dd'` format.

**Function Specifications:**
- The function should take a list of strings as input.
- Each string in the input list is formatted as `'yyyy-mm-dd hh:mm:ss'`.
- The function should return a list of strings where each element in the returned list contains only the date in the `'yyyy-mm-dd'` format.

In [10]:
### START FUNCTION
def date_parser(dates):
    """
    Function 3: Date Parser
    Date Parser function that takes a list of datetime strings and returns only the
    date in 'yyyy-mm-dd' format.
    The function returns the date only from a list of items.
    The input is a list of datetime strings and it outputs a list of dates.
    The dates variable is a list of dates represented as strings.
    The string contains the date in 'yyyy-mm-dd' format and the time in hh:mm:ss foramt.
    The first three entries in this variable are:
    dates[:3] == [
        '2019-11-29 12:50:54',
        '2019-11-29 12:46:53',
        '2019-11-29 12:46:10'
        ]
    Function Specifications:
    The function should take a list of strings as input.
    Each string in the input list is formatted as 'yyyy-mm-dd hh:mm:ss'.
    The function should return a list of strings where each element in the returned
    list contains only the date in the 'yyyy-mm-dd' format.
    Arg:
    A list of datetime strings
    Return:
    A date or list of dates
    Examples:
    Input
    [ '2019-11-29 12:50:54' , '2019-11-29 12:46:53' , '2019-11-29 12:46:10']
    Output
    [ '2019-11-29' , '2019-11-29' , '2019-11-29' ]
    """

    result = []                       # Define an empty list
    for i in range(len(dates)):
        res = dates[i][:10]           # Accessing first 10 elements of string
        result.append(res)           # Inserts the new results into a list
    return result
### END FUNCTION

In [11]:
date_parser(dates[:3]) == ['2019-11-29', '2019-11-29', '2019-11-29']
date_parser(dates[-3:]) == ['2019-11-20', '2019-11-20', '2019-11-20']

True

_**Expected Output:**_

```python
date_parser(dates[:3]) == ['2019-11-29', '2019-11-29', '2019-11-29']
date_parser(dates[-3:]) == ['2019-11-20', '2019-11-20', '2019-11-20']
```

## Function 4: Municipality & Hashtag Detector

Write a function which takes in a pandas dataframe and returns a modified dataframe that includes two new columns that contain information about the municipality and hashtag of the tweet.

**Function Specifications:**
* Function should take a pandas `dataframe` as input.
* Extract the municipality from a tweet using the `mun_dict` dictonary given below, and insert the result into a new column named `'municipality'` in the same dataframe.
* Use the entry `np.nan` when a municipality is not found.
* Extract a list of hashtags from a tweet into a new column named `'hashtags'` in the same dataframe.
* Use the entry `np.nan` when no hashtags are found.

**Hint:** you will need to `mun_dict` variable defined at the top of this notebook.

```

In [12]:
### START FUNCTION
def extract_municipality_hashtags(df):
    '''pandas dataframe and returns a modified dataframe that includes two new
    columns that contain information about the municipality and hashtag of the tweet.
    parameter
    ---------
      df(DataFrame): pandas dataframe as input containing Twitter information
    returns
    -------
      (DataFrame): pandas dataframe similar to inputed df but with 2 appended columns
                   (named 'municipality' and 'hashtags'). The 2 columns contain identified
                   major metro municipality and hashtags of tweets, respectively. The
                   columns will contain np.nan entries where municipality or hashtags are
                   not found.
    Example
    -------
      >>> twitter_feed = pd.DataFrame({'Tweets':['#ESKOMFREESTATE #MEDIASTATEMENT: ESKOM PLANS ELECTRICITY SUPPLY INTERRUPTIONS',
                                       'RT @CityPowerJhb: #RandburgOutage power outage in Greater Randburg'],
                             'Date': ['2019-11-29 12:17:43', '2019-11-28 13:34:41']})
      >>> extract_municipality_hashtags(twitter_feed)
                                       Tweets                 Date   unicipality                            hashtags
      0    #ESKOMFREESTATE #MEDIASTATEMENT...  2019-11-29 12:17:43  NaN           [#eskomfreestate, #mediastatement]
      1  RT @CityPowerJhb: #RandburgOutage...  2019-11-28 13:34:41  Johannesburg                   [#randburgoutage]
    '''

    def checkat(lst):
        '''
        This function checks if all the @-mentions are included in dictionary of major metros
        and returns the identified metro.
        The function is used with ".apply" method to isolate major metro in @-mentions
        '''
        mun_present = False
        if lst == []:
            return np.nan
        else:
            for at in lst:
                for key in mun_dict.keys():
                    if key in at:
                        mun_present = True
                        return mun_dict[key]
            if not mun_present:
                return np.nan


    #create pandas.Series objects containing list of columns with all @-mentions and hashtags, respectively
    atlist_col = df['Tweets'].apply(lambda x: [word for word in x.split() if word[0]=='@'])
    hashtag_list_col = df['Tweets'].apply(lambda x: [word.lower() for word in x.split() if word[0]=='#'])


    df['municipality']= atlist_col.apply(checkat)
    df['hashtags'] = hashtag_list_col.apply(lambda x: np.nan if x==[] else x)

    return df

### END FUNCTION

In [13]:
extract_municipality_hashtags(twitter_df.copy())

,Tweets,Date,municipality,hashtags
0,@BongaDlulane Please send an email to mediades...,2019-11-29 12:50:54,NaN,NaN
1,@saucy_mamiie Pls log a call on 0860037566,2019-11-29 12:46:53,NaN,NaN
2,@BongaDlulane Query escalated to media desk.,2019-11-29 12:46:10,NaN,NaN
3,"Before leaving the office this afternoon, head...",2019-11-29 12:33:36,NaN,NaN
4,#ESKOMFREESTATE #MEDIASTATEMENT : ESKOM SUSPEN...,2019-11-29 12:17:43,NaN,"[#eskomfreestate, #mediastatement]"
...,...,...,...,...
195,Eskom's Visitors Centres’ facilities include i...,2019-11-20 10:29:07,NaN,NaN
196,#Eskom connected 400 houses and in the process...,2019-11-20 10:25:20,NaN,"[#eskom, #eskom, #poweringyourworld]"
197,@ArthurGodbeer Is the power restored as yet?,2019-11-20 10:07:59,NaN,NaN
198,@MuthambiPaulina @SABCNewsOnline @IOL @eNCA @e...,2019-11-20 10:07:41,NaN,NaN


_**Expected Outputs:**_ 

```python

extract_municipality_hashtags(twitter_df.copy())

```
> <table class="dataframe" border="1">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Tweets</th>
      <th>Date</th>
      <th>municipality</th>
      <th>hashtags</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>@BongaDlulane Please send an email to mediades...</td>
      <td>2019-11-29 12:50:54</td>
      <td>NaN</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>1</th>
      <td>@saucy_mamiie Pls log a call on 0860037566</td>
      <td>2019-11-29 12:46:53</td>
      <td>NaN</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>2</th>
      <td>@BongaDlulane Query escalated to media desk.</td>
      <td>2019-11-29 12:46:10</td>
      <td>NaN</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Before leaving the office this afternoon, head...</td>
      <td>2019-11-29 12:33:36</td>
      <td>NaN</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>4</th>
      <td>#ESKOMFREESTATE #MEDIASTATEMENT : ESKOM SUSPEN...</td>
      <td>2019-11-29 12:17:43</td>
      <td>NaN</td>
      <td>[#eskomfreestate, #mediastatement]</td>
    </tr>
    <tr>
      <th>...</th>
      <td>...</td>
      <td>...</td>
      <td>...</td>
      <td>...</td>
    </tr>
    <tr>
      <th>195</th>
      <td>Eskom's Visitors Centresâ€™ facilities include i...</td>
      <td>2019-11-20 10:29:07</td>
      <td>NaN</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>196</th>
      <td>#Eskom connected 400 houses and in the process...</td>
      <td>2019-11-20 10:25:20</td>
      <td>NaN</td>
      <td>[#eskom, #eskom, #poweringyourworld]</td>
    </tr>
    <tr>
      <th>197</th>
      <td>@ArthurGodbeer Is the power restored as yet?</td>
      <td>2019-11-20 10:07:59</td>
      <td>NaN</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>198</th>
      <td>@MuthambiPaulina @SABCNewsOnline @IOL @eNCA @e...</td>
      <td>2019-11-20 10:07:41</td>
      <td>NaN</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>199</th>
      <td>RT @GP_DHS: The @GautengProvince made a commit...</td>
      <td>2019-11-20 10:00:09</td>
      <td>NaN</td>
      <td>NaN</td>
    </tr>
  </tbody>
</table>

## Function 5: Number of Tweets per Day

Write a function which calculates the number of tweets that were posted per day. 

**Function Specifications:**
- It should take a pandas dataframe as input.
- It should return a new dataframe, grouped by day, with the number of tweets for that day.
- The index of the new dataframe should be named `Date`, and the column of the new dataframe should be `'Tweets'`, corresponding to the date and number of tweets, respectively.
- The date should be formated as `yyyy-mm-dd`, and should be a datetime object. **Hint:** look up `pd.to_datetime` to see how to do this.

In [18]:
### START FUNCTION
def number_of_tweets_per_day(df):
    ''' Function which counts the number of tweets that were posted per day
    parameter
    ---------
      df(DataFrame): pandas dataframe as input containing Twitter information
    returns
    -------
      (DataFrame): New dataframe, grouped by day, with the number of tweets for that day. The dataframe is
                   indexed by Date which tweets were, and the column named 'Tweets' which corresponds
                   number of tweets.
    Example
    -------
      >>> twitter_feed = pd.DataFrame({'Tweets':['#ESKOMFREESTATE...', 'RT @CityPowerJhb...', 'Guys load..'],
                               'Date': ['2019-11-29 12:17:43', '2019-11-28 13:34:41', '2020-02-19 07:00:00']})
      >>> number_of_tweets_per_day(twitter_feed)
                  Tweets
            Date
      2019-11-29       2
      2020-02-19       1
      
    '''
    
    df = pd.to_datetime(df['Date'], yearfirst=True).apply(lambda x: x.date()).value_counts().to_frame()
    df = df.rename(columns={'Date':'Tweets'})
    df.index.name = 'Date'
    df.sort_index(inplace=True)
    
    return df

### END FUNCTION

In [19]:
number_of_tweets_per_day(twitter_df.copy())

,Tweets
Date,
2019-11-20,18
2019-11-21,11
2019-11-22,25
2019-11-23,19
2019-11-24,14
2019-11-25,20
2019-11-26,32
2019-11-27,13
2019-11-28,32


_**Expected Output:**_

```python

number_of_tweets_per_day(twitter_df.copy())

```

> <table class="dataframe" border="1">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Tweets</th>
    </tr>
    <tr>
      <th>Date</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>2019-11-20</th>
      <td>18</td>
    </tr>
    <tr>
      <th>2019-11-21</th>
      <td>11</td>
    </tr>
    <tr>
      <th>2019-11-22</th>
      <td>25</td>
    </tr>
    <tr>
      <th>2019-11-23</th>
      <td>19</td>
    </tr>
    <tr>
      <th>2019-11-24</th>
      <td>14</td>
    </tr>
    <tr>
      <th>2019-11-25</th>
      <td>20</td>
    </tr>
    <tr>
      <th>2019-11-26</th>
      <td>32</td>
    </tr>
    <tr>
      <th>2019-11-27</th>
      <td>13</td>
    </tr>
    <tr>
      <th>2019-11-28</th>
      <td>32</td>
    </tr>
    <tr>
      <th>2019-11-29</th>
      <td>16</td>
    </tr>
  </tbody>
</table>

# Function 6: Word Splitter

Write a function which splits the sentences in a dataframe's column into a list of the separate words. The created lists should be placed in a column named `'Split Tweets'` in the original dataframe. This is also known as [tokenization](https://www.geeksforgeeks.org/nlp-how-tokenizing-text-sentence-words-works/).

**Function Specifications:**
- It should take a pandas dataframe as an input.
- The dataframe should contain a column, named `'Tweets'`.
- The function should split the sentences in the `'Tweets'` into a list of seperate words, and place the result into a new column named `'Split Tweets'`. The resulting words must all be lowercase!
- The function should modify the input dataframe directly.
- The function should return the modified dataframe.

In [20]:
### START FUNCTION
def word_splitter(df):
    """ Splits sentences in a dataframe's column and returns a list of the separated words in lowercase
    Parameters:
    -----------
        df (dataframe): Dataframe of sentences in string format
    Returns:
    -----------
        (list): List of the seperated words in lowercase
     Examples
    -----------
    >>> word_splitter(df)
    @BongaDlulane Please send an email to mediades = [@bongadlulane, please, send, an, email to, mediades]
    """
    df['Split Tweets'] = list(map(lambda i: i.lower().split(),df.Tweets))
    return df

### END FUNCTION

In [21]:
word_splitter(twitter_df.copy())

,Tweets,Date,Split Tweets
0,@BongaDlulane Please send an email to mediades...,2019-11-29 12:50:54,"[@bongadlulane, please, send, an, email, to, m..."
1,@saucy_mamiie Pls log a call on 0860037566,2019-11-29 12:46:53,"[@saucy_mamiie, pls, log, a, call, on, 0860037..."
2,@BongaDlulane Query escalated to media desk.,2019-11-29 12:46:10,"[@bongadlulane, query, escalated, to, media, d..."
3,"Before leaving the office this afternoon, head...",2019-11-29 12:33:36,"[before, leaving, the, office, this, afternoon..."
4,#ESKOMFREESTATE #MEDIASTATEMENT : ESKOM SUSPEN...,2019-11-29 12:17:43,"[#eskomfreestate, #mediastatement, :, eskom, s..."
...,...,...,...
195,Eskom's Visitors Centres’ facilities include i...,2019-11-20 10:29:07,"[eskom's, visitors, centres’, facilities, incl..."
196,#Eskom connected 400 houses and in the process...,2019-11-20 10:25:20,"[#eskom, connected, 400, houses, and, in, the,..."
197,@ArthurGodbeer Is the power restored as yet?,2019-11-20 10:07:59,"[@arthurgodbeer, is, the, power, restored, as,..."
198,@MuthambiPaulina @SABCNewsOnline @IOL @eNCA @e...,2019-11-20 10:07:41,"[@muthambipaulina, @sabcnewsonline, @iol, @enc..."


_**Expected Output**_:

```python

word_splitter(twitter_df.copy()) 

```

> <table class="dataframe" border="1">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Tweets</th>
      <th>Date</th>
      <th>Split Tweets</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>@BongaDlulane Please send an email to mediades...</td>
      <td>2019-11-29 12:50:54</td>
      <td>[@bongadlulane, please, send, an, email, to, m...</td>
    </tr>
    <tr>
      <th>1</th>
      <td>@saucy_mamiie Pls log a call on 0860037566</td>
      <td>2019-11-29 12:46:53</td>
      <td>[@saucy_mamiie, pls, log, a, call, on, 0860037...</td>
    </tr>
    <tr>
      <th>2</th>
      <td>@BongaDlulane Query escalated to media desk.</td>
      <td>2019-11-29 12:46:10</td>
      <td>[@bongadlulane, query, escalated, to, media, d...</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Before leaving the office this afternoon, head...</td>
      <td>2019-11-29 12:33:36</td>
      <td>[before, leaving, the, office, this, afternoon...</td>
    </tr>
    <tr>
      <th>4</th>
      <td>#ESKOMFREESTATE #MEDIASTATEMENT : ESKOM SUSPEN...</td>
      <td>2019-11-29 12:17:43</td>
      <td>[#eskomfreestate, #mediastatement, :, eskom, s...</td>
    </tr>
    <tr>
      <th>...</th>
      <td>...</td>
      <td>...</td>
      <td>...</td>
    </tr>
    <tr>
      <th>195</th>
      <td>Eskom's Visitors Centresâ€™ facilities include i...</td>
      <td>2019-11-20 10:29:07</td>
      <td>[eskom's, visitors, centresâ€™, facilities, incl...</td>
    </tr>
    <tr>
      <th>196</th>
      <td>#Eskom connected 400 houses and in the process...</td>
      <td>2019-11-20 10:25:20</td>
      <td>[#eskom, connected, 400, houses, and, in, the,...</td>
    </tr>
    <tr>
      <th>197</th>
      <td>@ArthurGodbeer Is the power restored as yet?</td>
      <td>2019-11-20 10:07:59</td>
      <td>[@arthurgodbeer, is, the, power, restored, as,...</td>
    </tr>
    <tr>
      <th>198</th>
      <td>@MuthambiPaulina @SABCNewsOnline @IOL @eNCA @e...</td>
      <td>2019-11-20 10:07:41</td>
      <td>[@muthambipaulina, @sabcnewsonline, @iol, @enc...</td>
    </tr>
    <tr>
      <th>199</th>
      <td>RT @GP_DHS: The @GautengProvince made a commit...</td>
      <td>2019-11-20 10:00:09</td>
      <td>[rt, @gp_dhs:, the, @gautengprovince, made, a,...</td>
    </tr>
  </tbody>
</table>

# Function 7: Stop Words

Write a function which removes english stop words from a tweet.

**Function Specifications:**
- It should take a pandas dataframe as input.
- Should tokenise the sentences according to the definition in function 6. Note that function 6 **cannot be called within this function**.
- Should remove all stop words in the tokenised list. The stopwords are defined in the `stop_words_dict` variable defined at the top of this notebook.
- The resulting tokenised list should be placed in a column named `"Without Stop Words"`.
- The function should modify the input dataframe.
- The function should return the modified dataframe.


In [22]:
### START FUNCTION
def stop_words_remover(df):
    ''' Function counts removes english stop words from a tweet.
    parameter
    ---------
      df(DataFrame): pandas dataframe as input containing Twitter information
    returns
    -------
      (DataFrame): df DataFrame with additional column called 'Without Stop Words'. The column contains 
                   tokenised Tweets without stop-words and tokenized tweet is then added to column within
                   a list. 
    
    Example
    -------
    >>> twitter_feed = pd.DataFrame({'Tweets':['Cremora above the fridge', 'We'er almost done', 'Good work guys'],
                             'Date': ['2019-11-29 12:17:43', '2019-11-28 13:34:41', '2020-02-19 07:00:00']})
    >>> stop_words_remover(twitter_feed)
    
               
                         Tweets                  Date  Without Stop Words
    0  Cremora above the fridge   2019-11-29 12:17:43 	[cremora, fridge]  
    1         We'er almost done   2019-11-28 13:34:41             [we'er]
    2            Good work guys	  2020-02-19 07:00:00  [good, work, guys]
    '''

    splitter = df['Tweets'].apply(lambda x: x.lower().split())
    df['Without Stop Words'] = splitter.apply(lambda x: [word for word in x if word not in stop_words_dict['stopwords']])

    return df
### END FUNCTION

In [23]:
stop_words_remover(twitter_df.copy()).loc[0, "Without Stop Words"] == ['@bongadlulane', 'send', 'email', 'mediadesk@eskom.co.za']
stop_words_remover(twitter_df.copy()).loc[100, "Without Stop Words"] == ['#eskomnorthwest', '#mediastatement', ':', 'notice', 'supply', 'interruption', 'lichtenburg', 'area', 'https://t.co/7hfwvxllit']

True

_**Expected Output**_:

Specific rows:

```python
stop_words_remover(twitter_df.copy()).loc[0, "Without Stop Words"] == ['@bongadlulane', 'send', 'email', 'mediadesk@eskom.co.za']
stop_words_remover(twitter_df.copy()).loc[100, "Without Stop Words"] == ['#eskomnorthwest', '#mediastatement', ':', 'notice', 'supply', 'interruption', 'lichtenburg', 'area', 'https://t.co/7hfwvxllit']
```

Whole table:
```python
stop_words_remover(twitter_df.copy())
```

> <table class="dataframe" border="1">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Tweets</th>
      <th>Date</th>
      <th>Without Stop Words</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>@BongaDlulane Please send an email to mediades...</td>
      <td>2019-11-29 12:50:54</td>
      <td>[@bongadlulane, send, email, mediadesk@eskom.c...</td>
    </tr>
    <tr>
      <th>1</th>
      <td>@saucy_mamiie Pls log a call on 0860037566</td>
      <td>2019-11-29 12:46:53</td>
      <td>[@saucy_mamiie, pls, log, 0860037566]</td>
    </tr>
    <tr>
      <th>2</th>
      <td>@BongaDlulane Query escalated to media desk.</td>
      <td>2019-11-29 12:46:10</td>
      <td>[@bongadlulane, query, escalated, media, desk.]</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Before leaving the office this afternoon, head...</td>
      <td>2019-11-29 12:33:36</td>
      <td>[leaving, office, afternoon,, heading, weekend...</td>
    </tr>
    <tr>
      <th>4</th>
      <td>#ESKOMFREESTATE #MEDIASTATEMENT : ESKOM SUSPEN...</td>
      <td>2019-11-29 12:17:43</td>
      <td>[#eskomfreestate, #mediastatement, :, eskom, s...</td>
    </tr>
    <tr>
      <th>...</th>
      <td>...</td>
      <td>...</td>
      <td>...</td>
    </tr>
    <tr>
      <th>195</th>
      <td>Eskom's Visitors Centresâ€™ facilities include i...</td>
      <td>2019-11-20 10:29:07</td>
      <td>[eskom's, visitors, centresâ€™, facilities, incl...</td>
    </tr>
    <tr>
      <th>196</th>
      <td>#Eskom connected 400 houses and in the process...</td>
      <td>2019-11-20 10:25:20</td>
      <td>[#eskom, connected, 400, houses, process, conn...</td>
    </tr>
    <tr>
      <th>197</th>
      <td>@ArthurGodbeer Is the power restored as yet?</td>
      <td>2019-11-20 10:07:59</td>
      <td>[@arthurgodbeer, power, restored, yet?]</td>
    </tr>
    <tr>
      <th>198</th>
      <td>@MuthambiPaulina @SABCNewsOnline @IOL @eNCA @e...</td>
      <td>2019-11-20 10:07:41</td>
      <td>[@muthambipaulina, @sabcnewsonline, @iol, @enc...</td>
    </tr>
    <tr>
      <th>199</th>
      <td>RT @GP_DHS: The @GautengProvince made a commit...</td>
      <td>2019-11-20 10:00:09</td>
      <td>[rt, @gp_dhs:, @gautengprovince, commitment, e...</td>
    </tr>
  </tbody>
</table>